# Advent of Code 2016
[see here](http://adventofcode.com/2016)

## Preparation
This is inspired by [Peter Norvigs solution](https://github.com/norvig/pytudes/blob/master/ipynb/Advent%20of%20Code.ipynb)

In [1]:
# Python 3.x
import re
import numpy as np
import math
import urllib.request

from collections import Counter, defaultdict, namedtuple, deque
from functools   import lru_cache, reduce
from itertools   import permutations, combinations, chain, cycle, product, islice
from heapq       import heappop, heappush

def Input(day):
    "Open this day's input file."
    
    filename = 'input/input{}.txt'.format(day)
    try:
        with open(filename, 'r') as f:
            text = f.read()
        return text
    except FileNotFoundError:
        url = 'http://adventofcode.com/2016/day/{}/input'.format(day)
        print('input file not found. opening browser...')
        import webbrowser
        webbrowser.open(url)

def fs(*items): return frozenset(items)
cat = ''.join
first = lambda x: list(x)[0]

def shift(it, n):
    return it[n:] + it[:n]

def rot(original,clockwise=True):
    '''rotate 2D matrix'''
    if clockwise:
        return list(zip(*original[::-1]))
    else:
        return list(zip(*original[::-1]))[::-1]

def prod(it):
    return reduce(lambda x,y: x*y, it)

def grep(pattern, lines):
    "Print lines that match pattern."
    for line in lines:
        if re.search(pattern, line):
            print(line)

def groupby(iterable, key=lambda it: it):
    "Return a dic whose keys are key(it) and whose values are all the elements of iterable with that key."
    dic = defaultdict(list)
    for it in iterable:
        dic[key(it)].append(it)
    return dic

def powerset(iterable):
    "Yield all subsets of items."
    items = list(iterable)
    for r in range(len(items)+1):
        for c in combinations(items, r):
            yield c

def neighbors4(point): 
    "The four neighbors (without diagonals)."
    x, y = point
    return ((x+1, y), (x-1, y), (x, y+1), (x, y-1))

def neighbors8(point): 
    "The eight neighbors (with diagonals)."
    x, y = point 
    return ((x+1, y), (x-1, y), (x, y+1), (x, y-1),
            (X+1, y+1), (x-1, y-1), (x+1, y-1), (x-1, y+1))

def cityblock_distance(p, q=(0, 0)): 
    "City block distance between two points."
    return abs(p[0] - q[0]) + abs(p[1] - q[1])

def euclidean_distance(p, q=(0, 0)): 
    "Euclidean (hypotenuse) distance between two points."
    return math.hypot(p[0] - q[0], p[1] - q[1])

def locate2D(m, val):
    '''locate value in 2D list'''
    for i, line in enumerate(m):
        j=-1
        try:
            j = line.index(val)
        except ValueError:
            continue
        break
    else:
        i = -1
    return (i,j)

def trace1(f):
    "Print a trace of the input and output of a function on one line."
    def traced_f(*args):
        result = f(*args)
        print('{}({}) = {}'.format(f.__name__, ', '.join(map(str, args)), result))
        return result
    return traced_f

def astar_search(start, h_func, moves_func):
    "Find a shortest sequence of states from start to a goal state (a state s with h_func(s) == 0)."
    frontier  = [(h_func(start), start)] # A priority queue, ordered by path length, f = g + h
    previous  = {start: None}  # start state has no previous state; other states will
    path_cost = {start: 0}     # The cost of the best path to a state.
    while frontier:
        (f, s) = heappop(frontier)
        if h_func(s) == 0:
            return Path(previous, s)
        for s2 in moves_func(s):
            new_cost = path_cost[s] + 1
            if s2 not in path_cost or new_cost < path_cost[s2]:
                heappush(frontier, (new_cost + h_func(s2), s2))
                path_cost[s2] = new_cost
                previous[s2] = s
    return dict(fail=True, front=len(frontier), prev=len(previous))

def bfs(start, moves_func, dest=None, max_step=None):
    frontier  = [(0, start)] # A priority queue, ordered by path length, f = g + h
    previous  = {start: None}  # start state has no previous state; other states will
    path_cost = {start: 0}     # The cost of the best path to a state.
    while frontier:
        (f, s) = heappop(frontier)
        if s == dest:
            return Path(previous, s)
        if f == max_step:
            break
        for s2 in moves_func(s):
            new_cost = path_cost[s] + 1
            if s2 not in path_cost or new_cost < path_cost[s2]:
                heappush(frontier, (new_cost, s2))
                path_cost[s2] = new_cost
                previous[s2] = s
    return dict(fail=True, front=len(frontier), prev=len(previous), cost=path_cost)
    
    
                
def Path(previous, s): 
    "Return a list of states that lead to state s, according to the previous dict."
    return ([] if (s is None) else Path(previous, previous[s]) + [s])

## Day 1

In [ ]:
Point = complex             
N, S, E, W = 1j, -1j, 1, -1 # Unit vectors for headings

def distance(point): 
    "City block distance between point and the origin."
    return abs(point.real) + abs(point.imag)

def how_far(moves):
    "After following moves, how far away from the origin do we end up?"
    loc, heading = 0, N # Begin at origin, heading North
    for (turn, dist) in parse(moves):
        heading *= turn
        loc += heading * dist
    return distance(loc)

def parse(text):
    "Return a list of (turn, distance) pairs from text of form 'R2, L42, ...'"
    turns = dict(L=N, R=S)
    return [(turns[RL], int(d))
           for (RL, d) in re.findall(r'(R|L)(\d+)', text)]

assert distance(Point(3, 4)) == 7 # City block distance; Euclidean distance would be 5
assert parse('R2, L42') == [(S, 2), (N, 42)]
assert how_far("R2, L3") == 5
assert how_far("R2, R2, R2") == 2
assert how_far("R5, L5, R5, R3") == 12
how_far(Input(1))

In [ ]:
"""Part 2. first place that is visited twice."""

Point = complex             
N, S, E, W = 1j, -1j, 1, -1 # Unit vectors for headings

def distance(point): 
    "City block distance between point and the origin."
    return abs(point.real) + abs(point.imag)

def how_far(moves):
    "After following moves, how far away from the origin do we end up?"
    visited = set()
    loc, heading = 0, N # Begin at origin, heading North
    for (turn, dist) in parse(moves):
        heading *= turn
        for _ in range(dist):
            loc += heading
            if loc in visited:
                return distance(loc)
            else:
                visited.add(loc)
    return distance(loc)

def parse(text):
    "Return a list of (turn, distance) pairs from text of form 'R2, L42, ...'"
    turns = dict(L=N, R=S)
    return [(turns[RL], int(d))
           for (RL, d) in re.findall(r'(R|L)(\d+)', text)]

how_far(Input(1))

## Day 2

In [ ]:
Keypad = str.split

keypad = Keypad("""
.....
.123.
.456.
.789.
.....
""")

assert keypad[2][2] == '5'

off = '.'

def decode(instructions, x=2, y=2):
    """Follow instructions, keeping track of x, y position, and
    yielding the key at the end of each line of instructions."""
    for line in instructions:
        for C in line:
            x, y = move(C, x, y)
        yield keypad[y][x]

def move(C, x, y):
    "Make the move corresponding to this character (L/R/U/D)"
    if   C == 'L' and keypad[y][x-1] is not off: x -= 1
    elif C == 'R' and keypad[y][x+1] is not off: x += 1
    elif C == 'U' and keypad[y-1][x] is not off: y -= 1
    elif C == 'D' and keypad[y+1][x] is not off: y += 1
    return x, y

assert move('U', 2, 2) == (2, 1)
assert move('U', 2, 1) == (2, 1)
assert cat(decode("ULL RRDDD LURDL UUUUD".split())) == '1985'

cat(decode(Input(2).strip().split('\n')))

In [ ]:
keypad = Keypad("""
.......
...1...
..234..
.56789.
..ABC..
...D...
.......
""")

assert keypad[3][1] == '5'

cat(decode(Input(2).strip().split('\n'), x=1, y=3))

## Day 3

In [ ]:
def is_triangle(sides):
    a,b,c = sorted(sides)
    return a+b>c and a+c>b and b+c>a

def parse(text):
    triangle_list = []
    for line in text.split('\n'):
        sides = tuple(int(word) for word in re.findall(r'\S+',line))
        if sides:
            triangle_list.append(sides)
    return triangle_list

triangles = parse(Input(3))
print('part 1: ' + str(len([tri for tri in triangles if is_triangle(tri)])))
from itertools import chain, zip_longest
triangles2 = chain(*zip(*triangles))
def grouper(iterable, n, fillvalue=None):
    "Collect data into fixed-length chunks or blocks"
    # grouper('ABCDEFG', 3, 'x') --> ABC DEF Gxx
    args = [iter(iterable)] * n
    return zip_longest(fillvalue=fillvalue, *args)
triangles2 = grouper(triangles2,3)
print('part 2: ' + str(len([tri for tri in triangles2 if is_triangle(tri)])))

## Day 4 

In [ ]:
from collections import Counter
rooms = Input(4).strip().split('\n')
pattern = re.compile(r'(?P<name>[-a-z]+)-(?P<sector>\d+)\[(?P<check>[a-z]+)\]')

def parse_room(room):
    return pattern.match(room).groupdict()

def check_room(room):
    c = Counter(room['name'].replace('-',''))
    check = cat(sorted(c, key=lambda x: (-c[x],x))[:5])
    return check == room['check']
    
rooms = [parse_room(room) for room in rooms]
rooms = [room for room in rooms if check_room(room)]
sum(int(room['sector']) for room in rooms)

In [ ]:
def alpha_rot(n):
    return cat(chr(ord('a')+(i+n)%26) for i in range(26))
def decrypt_name(n):
    t = str.maketrans(alpha_rot(0)+'-', alpha_rot(n)+' ')
    return lambda s: s.translate(t)
assert decrypt_name(343)('qzmt-zixmtkozy-ivhz') == 'very encrypted name'
for room in rooms:
    sector = int(room['sector'])
    dec = decrypt_name(sector)
    dec_name = dec(room['name'])
    if 'pole' in dec_name:
        print(room['sector'] + ' ' + dec_name)

## Day 5

In [ ]:
door_id = 'abbhdwsy'
import hashlib
def door_md5(door_id, i):
    m = hashlib.md5()
    m.update(bytes(door_id + str(i),'utf-8'))
    return m.hexdigest()

assert door_md5('abc',3231929)[:6] == '000001'

def gen_pwd(door_id):
    pwd = ''
    i = 0
    while len(pwd) < 8:
        h = door_md5(door_id, i)
        if h.startswith('00000'):
            pwd += h[5]
        i+=1
    return pwd

#assert gen_pwd('abc') == '18f47a30'
gen_pwd(door_id)

In [ ]:
def gen_pwd2(door_id):
    pwd = '_'*8
    i = 0
    print(pwd)
    while '_' in pwd:
        h = door_md5(door_id, i)
        pos = int(h[5],base=16)
        if h.startswith('00000') and pos < 8 and pwd[pos] == '_':
            pwd = pwd[:pos]+ h[6] + pwd[pos+1:]
            print(pwd)
        i+=1
    return pwd
gen_pwd2(door_id)

## Day 6

In [ ]:
msgs = Input(6).split()
cols = list(zip(*msgs))
from collections import Counter
message = cat(Counter(col).most_common(1)[0][0] for col in cols)
print(message)
message2 = cat(Counter(col).most_common()[-1][0] for col in cols)
print(message2)

## Day 7

In [ ]:
ips = Input(7).split()
abba = re.compile(r'(\w)(?!\1)(\w)\2\1')
split = re.compile(r'(\w+)+')

def check_TLS(ip):
    parts = split.findall(ip)
    good_abba = False
    bad_abba = False
    for i, part in enumerate(parts):
        if abba.search(part):
            if i%2: #0-> outside of []. 1-> inside of []
                bad_abba = True
            else:
                good_abba = True
    return good_abba and not bad_abba

assert check_TLS('abba[mnop]qrst') == True
assert check_TLS('abcd[bddb]xyyx') == False
assert check_TLS('aaaa[qwer]tyui') == False
assert check_TLS('ioxxoj[asdfgh]zxcvbn') == True

def check_SSL(ip):
    pass

assert check_SSL('aba[bab]xyz')
assert check_SSL('xyx[xyx]xyx') == False
assert check_SSL('aaa[kek]eke')
assert check_SSL('zazbz[bzb]cdb')

ips = [ip for ip in ips if check_TLS(ip)]
len(ips)


## Day 8
Format for instructions:
```
rect 2x1
rotate row y=0 by 5
rotate column x=0 by 1
```

In [ ]:
instructions = Input(8).splitlines()
rect = re.compile(r'rect (?P<width>\d+)x(?P<height>\d+)')
rotr = re.compile(r'rotate row y=(?P<row>\d+) by (?P<shift>\d+)')
rotc = re.compile(r'rotate column x=(?P<col>\d+) by (?P<shift>\d+)')

def get_param(pattern, text):
    match = pattern.findall(text)
    if not match: return
    return tuple(int(x) for x in match[0])

FILLCHAR = '@'
WIDTH, HEIGHT = 50,6
disp = [[' ' for _ in range(WIDTH)] for _ in range(HEIGHT)]
for line in instructions:
    param = get_param(rect,line)
    if param:
        width, height = param
        for i, j in product(range(width), range(height)):
            disp[j][i] = FILLCHAR
        continue
    param = get_param(rotr, line)
    if param:
        row, shift = param
        disp[row] = disp[row][-shift:] + disp[row][:-shift]
        continue
    param = get_param(rotc, line)
    if param:
        disp = list(zip(*disp))
        row, shift = param
        disp[row] = disp[row][-shift:] + disp[row][:-shift]
        disp = list(list(row) for row in zip(*disp))
        continue
dispstr = '\n'.join(cat(line) for line in disp)
print('#Pixels lit: '+ str(dispstr.count(FILLCHAR)))
print('Password:')
print(dispstr)

## Day 9

In [ ]:
ctext = Input(9).strip()
mark = re.compile(r'\((?P<len>\d+)x(?P<rep>\d+)\)')
def decompress(ctext):
    dtext = ''
    pos = 0
    while pos < len(ctext):
        m = mark.search(ctext, pos)
        if not m:
            dtext += ctext[pos:]
            pos = len(ctext)
            break
        start = m.start()
        end = m.end()
        length, repeat = int(m.groupdict()['len']), int(m.groupdict()['rep'])
        dtext += ctext[pos:start]
        dtext += ctext[end:end+length] * repeat
        pos = end+length
    return dtext

assert decompress('ADVENT') == 'ADVENT'
assert decompress('A(1x5)BC') == 'ABBBBBC'
assert decompress('(3x3)XYZ') == 'XYZXYZXYZ'
assert decompress('A(2x2)BCD(2x2)EFG') == 'ABCBCDEFEFG'
assert decompress('(6x1)(1x3)A') == '(1x3)A'
assert decompress('X(8x2)(3x3)ABCY') == 'X(3x3)ABC(3x3)ABCY'


dtext = decompress(ctext)
print(dtext[:200] + ' ...')
print('Total Length: ' + str(len(dtext)))

In [ ]:
def decompress2len(ctext):
    dtext = 0 #is now length
    pos = 0
    while pos < len(ctext):
        m = mark.search(ctext, pos)
        if not m:
            dtext += len(ctext) - pos
            pos = len(ctext)
            break
        start = m.start()
        end = m.end()
        length, repeat = int(m.groupdict()['len']), int(m.groupdict()['rep'])
        dtext += start-pos
        dtext += decompress2len(ctext[end:end+length]) * repeat
        pos = end+length
    return dtext


assert decompress2len('(27x12)(20x12)(13x14)(7x10)(1x12)A') == 241920
assert decompress2len('(25x3)(3x3)ABC(2x3)XY(5x2)PQRSTX(18x9)(3x2)TWO(5x7)SEVEN') == 445

dtext = decompress2len(ctext)
print('Total Length: ' + str(dtext))

## Day 10

In [ ]:
instr = Input(10).strip()
TARGET = set((17,61))
#print(log[:497] + '\n...')
pat_rec = re.compile(r'value (\d+) goes to (\w+ \d+)')
pat_giv = re.compile(r'(\w+ \d+) gives low to (\w+ \d+) and high to (\w+ \d+)')

def give(stash, recipient, value, target=None):
    stash[recipient].add(value)
    if len(stash[recipient]) == 2:
        s = stash[recipient]
        if s == target: print('{} has {}'.format(recipient,s))
        stash[recipient] = set()
        give(stash, flow[recipient][0], min(s),target)
        give(stash, flow[recipient][1], max(s),target)

flow = {giver: (low, high) for giver, low, high in pat_giv.findall(instr)}
stash = defaultdict(set)
for val, dest in pat_rec.findall(instr):
    give(stash, dest, int(val),TARGET)

outputs = ['output '+str(i) for i in range(3)]
print('{} = {}'.format(' * '.join(outputs), prod(stash[o].pop() for o in outputs)))

## Day 11

In [ ]:
locs = Input(11).strip()
#print(locs)
loc_pat  = re.compile(r'The \w+ floor contains (.*)[.]')
none_pat = 'nothing relevant'
gen_pat  = re.compile(r'(\w+) generator')
chip_pat = re.compile(r'(\w+)-compatible microchip')

floors = dict()
elements = set()
state = dict()
for floor, content in enumerate(loc_pat.findall(locs)):
    floors[floor] = {'G':set(), 'M':set()}
    if content == none_pat:
        continue
    for element in gen_pat.findall(content):
        floors[floor]['G'].add(element)
        elements.add(element)
    for element in chip_pat.findall(content):
        floors[floor]['M'].add(element)
        elements.add(element)
legal_floors = set(floors.keys())
elements = {chr(ord('A')+i) : el for i,el in enumerate(sorted(elements))}
for floor in floors:
    state[floor] = fs(*([el+'G' for el,ell in elements.items() if ell in floors[floor]['G']]+
                        [el+'M' for el,ell in elements.items() if ell in floors[floor]['M']]))
    

state     

In [ ]:
def _el_info(floor,el): return '{} {}'.format(' M'[el in floor['M']], ' G'[el in floor['G']]) #helper    
def diagram(floors, elements, elevator=0):
    'will format and display a state'
    elements = sorted(elements)
    fstr = '{: <2} | {: <1} | '+' | '.join('{: ^'+str(max(len(e),4))+'}' for e in elements)
    header = fstr.format('F','E',*elements)
    print(header + '\n' + '-'*len(header))
    for floor in reversed(sorted(floors.keys())):
        el_info = [_el_info(floors[floor],el) for el in elements]
        print(fstr.format(floor, ' E'[floor==elevator], *el_info))

diagram(floors, elements.values(), 0)    

## Day 12

In [76]:
inst = Input(12).strip().splitlines()
#print('\n'.join('{: >4}: {}'.format(i, instr) for i,instr in enumerate(inst)))
#print('#'*10)
reg = {chr(ord('a')+i): 0 for i in range(4)}
def cpy(pos,x,y):
    if x in reg:
        reg[y] = reg[x]
    else:
        reg[y] = int(x)
    return pos+1
def inc(pos,r):
    reg[r] += 1
    return pos+1
def dec(pos,r):
    reg[r] -= 1
    return pos+1
def jnz(pos,x,y):
    if x in reg and reg[x] != 0:
        return pos + int(y)
    elif x not in reg and x != 0:
        return pos + int(y )
    else:
        return pos+1
cpy_pat = re.compile(r'cpy (\w+) (\w+)')
inc_pat = re.compile(r'inc (\w+)')
dec_pat = re.compile(r'dec (\w+)')
jnz_pat = re.compile(r'jnz (\w+) ([-+]?\w+)')
pos = 0
patterns = (cpy_pat, inc_pat, dec_pat, jnz_pat)
funs = (cpy, inc, dec, jnz)
while pos < len(inst):
    cmd = inst[pos]
    m = first(filter(lambda x: x[1] ,enumerate(pat.findall(cmd) for pat in patterns)))
    pos = funs[m[0]](pos,*m[1][0])
print('Part 1 - Value of Register a: '+str(reg['a']))    

Part 1 - Value of Register a: 318007


In [28]:
reg = {chr(ord('a')+i): 0 for i in range(4)}
reg['c'] = 1
pos=0
while pos < len(inst):
    cmd = inst[pos]
    m = first(filter(lambda x: x[1] ,enumerate(pat.findall(cmd) for pat in patterns)))
    pos = funs[m[0]](pos,*m[1][0])
print('Part 2 - Value of Register a: '+str(reg['a']))    

Part 2 - Value of Register a: 9227661


## Day 13

In [97]:
NUM = int(Input(13).strip())
DEST = 31,39
START = 1,1
def is_wall(point):
    x,y = point
    if x < 0 or y < 0: return True 
    return bin(x*x + 3*x + 2*x*y + y + y*y+NUM).count('1') % 2 != 0 

def moves_func(point):
    return [np for np in neighbors4(point) if not is_wall(np)]
def h_func(point):
    return cityblock_distance(point,DEST)

path = astar_search(START,h_func,moves_func)
xmax, ymax = list(map(max,zip(*path)))
maze = [[' █'[is_wall((x,y))] for x in range(-1,xmax+2)] for y in range(-1,ymax+2)]
for x,y in path:
    maze[y+1][x+1] = 'P'  # +1 since we draw the maze starting at "-1"
maze[START[1]+1][START[0]+1] = 'S'
maze[DEST[1]+1][DEST[0]+1] = 'D'
print('Partial Maze:\n' + '\n'.join(cat(line) for line in maze) + '\n\n█: wall, S: start, D: destination, P: path')
print('Minimum number of steps: '+str(len(path)-1)) # -1 because the start is included in path

Partial Maze:
██████████████████████████████████
████ ███   █ █  █ ██    █   ██ █  
█PS███ █   █  █    ██ █   █    █  
█P█  █ ██ ███████ █ █ ████ ██ ███ 
█PP█ █ ██ █    ██   █  █ ████ ████
██PP██ ██ █  █   ███ █     █     █
█ █P██  ████████ █ █████   █  ██ █
█  PPPPPPPPP ███     █████ █████  
███ ██ ████P  █ ██    █  █ █  █ ██
█ █  █ ██ █P█ ██████  █ ██  █ ████
████      █PP█  █  ████ ███  █  █ 
█████  ███ █P██ █ █   █  █ █ ██ ██
█ ██████ ███P ███  ██ █  ███  ██ █
█  ██  █ PPPP  ███ █  ███      █ █
█      ██P██ █     █    █ ██ █ ███
█████ █ █P██   ███ ██   ██ █  █  █
█  ██  ██P█ ███  █ ███ █ ██ █  █  
██ ███ █PP█   █ ██   █  █ █  █    
██  █  █P██████ █████ █  ███  ██ █
█   ██ █P██  █   ██ █ ██ ████  █ █
██   █  PPPP █      █     █ ███   
███ ███████P████████ ██ █ ███ ██  
███ █   ███P██PPP█ ██ █  █     ██ 
███ █     █PPPP█PP█ ██ █ ████ █ █ 
█  ███ ██ █ ███ █P██ ███   ██  ██ 
██  ██ █  ██  ██ P █     █  ██ █  
███    █   ███ █ P ██ █████ █  █  
██ ██ ███ █  █ ██P█ ███  █  ██ ███
█████ 

In [94]:
print('Number of reachable positions in 50 steps: ' + str(len(bfs(START, moves_func, max_step=50)['cost'])))

Number of reachable positions in 50 steps: 127


## Day 14

In [116]:
salt = Input(14).strip()
import hashlib
@lru_cache(maxsize=1000000)
def gen_hash(salt, i):
    m = hashlib.md5()
    m.update(bytes(salt + str(i),'utf-8'))
    return m.hexdigest()
pat3 = re.compile(r'(\w)\1\1')
pat5 = lambda s: re.compile(s+r'{5}')
hashes = []
i = 0
while len(hashes) < 64:
    current = gen_hash(salt,i)
    triples = pat3.findall(current)
    if triples:
        p5 = pat5(first(triples))
        for j in range(1,1001):
            if p5.findall(gen_hash(salt,i+j)):
                hashes.append(current)
                break
    i+=1
print('Part 1: found 64th hash at index '+str(i-1)+' : '+hashes[-1])

Part 1: found 64th hash at index 15035 : 3100d6c71100016ac85a3c243c87e3bb


In [118]:
@lru_cache(maxsize=1000000)
def gen_stretched_hash(salt, i):
    m = hashlib.md5()
    m.update(bytes(salt + str(i),'utf-8'))
    nh = m.hexdigest()
    for i in range(2016):
        m = hashlib.md5()
        m.update(bytes(nh,'utf-8'))
        nh = m.hexdigest()
    return nh
pat3 = re.compile(r'(\w)\1\1')
pat5 = lambda s: re.compile(s+r'{5}')
hashes = []
i = 0
while len(hashes) < 64:
    current = gen_stretched_hash(salt,i)
    triples = pat3.findall(current)
    if triples:
        p5 = pat5(first(triples))
        for j in range(1,1001):
            if p5.findall(gen_stretched_hash(salt,i+j)):
                hashes.append(current)
                break
    i+=1
print('Part 2: found 64th hash at index '+str(i-1)+' : '+hashes[-1])

Part 2: found 64th hash at index 19968 : 0f3cec9efe3b6c6f38f514f4fa87eee6


## Day 15

In [135]:
inst = Input(15).strip()
disc = namedtuple('disc', 'id numpos time curpos')
state = [disc(*map(int,re.compile(r'\d+').findall(line))) for line in inst.split('\n')]
def gen_pos(i,state):
    return [(d.curpos + d.id + i) % d.numpos for d in state]
i=0
while any(gen_pos(i,state)):
    i+=1
print('Part 1: aligned state after '+str(i)+' s')

Part 1: aligned state after 16824 s


In [136]:
state.append(disc(7,11,0,0))
i=0
while any(gen_pos(i,state)):
    i+=1
print('Part 2: aligned state after '+str(i)+' s')

Part 2: aligned state after 3543984 s


## Day 16

In [241]:
state = Input(16).strip()
DISK_SIZE = 272

def dragon(a):
    return a + '0' + cat(reversed(a)).replace('0','x').replace('1','0').replace('x','1')

def checksum(a):
    s = ''
    while a:
        p1, p2, *a = a
        if p1 == p2: s += '1'
        else: s+='0'
    return s
def checksum2(a):
    a1 = int(a[0::2],2)
    a2 = int(a[1::2].replace('0','x').replace('1','0').replace('x','1'),2)
    #print(bin(a2))
    #a2 ^=  2**(a2.bit_length()+1)-1
    #a2 = ~a2
    #print(bin(a2))
    return bin(a1^a2)[2:]

def gen_checksum(initial_state, disk_size):
    state = initial_state
    while len(state) < disk_size:
        state = dragon(state)
    state = state[:disk_size]
    print(state[:500])
    s = checksum(state)
    while len(s) % 2 == 0:
        s = checksum(s)
    return s

In [242]:
print('Part 1: checksum is '+str(gen_checksum(state, DISK_SIZE)))
print('Part 2: checksum is '+str(gen_checksum(state, 35651584)))

10001110011110000011110000110001110010001110011110000111110000110001110010001110011110000011110000110001110110001110011110000111110000110001110010001110011110000011110000110001110010001110011110000111110000110001110110001110011110000011110000110001110110001110011110000111
Part 1: checksum is 10010101010011101
10001110011110000011110000110001110010001110011110000111110000110001110010001110011110000011110000110001110110001110011110000111110000110001110010001110011110000011110000110001110010001110011110000111110000110001110110001110011110000011110000110001110110001110011110000111110000110001110010001110011110000011110000110001110010001110011110000111110000110001110010001110011110000011110000110001110110001110011110000111110000110001110110001110011110000011110000110001110010001110011110000111110000110001


KeyboardInterrupt: 

In [240]:
while len(state) < DISK_SIZE:
    state = dragon(state)
state = state[:DISK_SIZE]
print(len(state))
print(checksum(state))
print(checksum2(state))

272
0110010111111110110110010101111110110110010111111110100110010101111110110110010111111110110110010101111110100110010111111110100110010101
110010111111110110110010101111110110110010111111110100110010101111110110110010111111110110110010101111110100110010111111110100110010101


## Day  18

In [16]:
traps = Input(18).strip()
trap_parents = set(['^^.','.^^','^..','..^'])

def gen_room(initial_row, num_rows):
    rows = [initial_row]
    for i in range(num_rows-1):
        pr = '.' + rows[i] + '.' # prev row
        row = ''
        for triple in zip(pr[:-2],pr[1:-1],pr[2:]):
            row += '.^'[cat(triple) in trap_parents]
        rows.append(row)
    return rows

rows = gen_room(traps, 40)
print('The generated field of traps looks like this:\n\n' + '\n'.join(rows))
print('Day 18 part 1: there are {} safe tiles'.format(cat(rows).count('.')))

The generated field of traps looks like this:

^..^^.^^^..^^.^...^^^^^....^.^..^^^.^.^.^^...^.^.^.^.^^.....^.^^.^.^.^.^.^.^^..^^^^^...^.....^....^.
.^^^^.^.^^^^^..^.^^...^^..^...^^^.^.....^^^.^........^^^...^..^^...........^^^^^...^^.^.^...^.^..^.^
^^..^...^...^^^..^^^.^^^^^.^.^^.^..^...^^.^..^......^^.^^.^.^^^^^.........^^...^^.^^^....^.^...^^...
^^^^.^.^.^.^^.^^^^.^.^...^...^^..^^.^.^^^..^^.^....^^^.^^...^...^^.......^^^^.^^^.^.^^..^...^.^^^^..
^..^.......^^.^..^....^.^.^.^^^^^^^...^.^^^^^..^..^^.^.^^^.^.^.^^^^.....^^..^.^.^...^^^^.^.^..^..^^.
.^^.^.....^^^..^^.^..^......^.....^^.^..^...^^^.^^^^...^.^.....^..^^...^^^^^.....^.^^..^....^^.^^^^^
^^^..^...^^.^^^^^..^^.^....^.^...^^^..^^.^.^^.^.^..^^.^...^...^.^^^^^.^^...^^...^..^^^^.^..^^^.^...^
^.^^^.^.^^^.^...^^^^^..^..^...^.^^.^^^^^...^^....^^^^..^.^.^.^..^...^.^^^.^^^^.^.^^^..^..^^^.^..^.^.
..^.^...^.^..^.^^...^^^.^^.^.^..^^.^...^^.^^^^..^^..^^^.......^^.^.^..^.^.^..^...^.^^^.^^^.^..^^...^
.^...^.^...^^..^^^.^^.^.^^....^^^^..^.^^^.^.

In [17]:
%time rows = gen_room(traps, 400000)
print('Day 18 part 2: there are {} safe tiles'.format(cat(rows).count('.')))

Wall time: 13.7 s
Day 18 part 2: there are 19998750 safe tiles


## Day 20

In [13]:
ranges = Input(20).strip()
ranges = sorted([list(map(int,r.split('-'))) for r in ranges.splitlines()])
low = 0
for lo, hi in ranges:
    if lo <= low: low = max(hi + 1,low)
print('Day 20 part 1: lowest allowed IP is '+str(low))

Day 20 part 1: lowest allowed IP is 22887907


## Day 24
TSM with only 5040 posible paths. Fun.

In [285]:
maze = Input(24).strip()
poi = sorted(re.findall(r'\d+',maze))
maze = maze.split()
poi_coord = {i: locate2D(maze,i) for i in poi}
routes = [[poi[0]] + list(line) for line in permutations(poi[1:])] # always start at 0

def moves_func(point):
    return [(x,y) for x, y in neighbors4(point) if maze[x][y] != '#']

dists = dict()
for a, b in combinations(poi,2):
    hfunc = lambda point: cityblock_distance(point,poi_coord[b])
    path = astar_search(poi_coord[a],hfunc,moves_func)
    dists[fs(a, b)] = len(path) -1

costs = [(sum(dists[fs(a, b)] for a, b in zip(route[:-1],route[1:])), route) for route in routes]
cost, path = min(costs)
print('Day 24 part 1: the shortest route takes {} steps (visit order: {}).'.format(cost,path))

Day 24 part 1: the shortest route takes 498 steps (visit order: ['0', '1', '6', '4', '2', '3', '7', '5']).


In [286]:
routes = [[poi[0]] + list(line) + [poi[0]] for line in permutations(poi[1:])] # always start and end at 0
costs = [(sum(dists[fs(a, b)] for a, b in zip(route[:-1],route[1:])), route) for route in routes]
cost, path = min(costs)
print('Day 24 part 2: the shortest route that ends back at 0 takes {} steps (visit order: {}).'.format(cost,path))

Day 24 part 2: the shortest route that ends back at 0 takes 804 steps (visit order: ['0', '1', '2', '3', '7', '5', '4', '6', '0']).
